# Final Project: Tự tử tại Ấn Độ

Họ tên: Nguyễn Minh Tâm

MSSV: 1712746

In [3]:
#import 
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

[Main Ideas and inspiraiton (amulya9 notebook)](https://www.kaggle.com/amulya9/data-analysis-of-suicides-in-india)

## Khám phá dữ liệu bao quát

In [7]:
df = pd.read_csv('Suicides in India 2001-2012.csv')
df.head()

,State,Year,Type_code,Type,Gender,Age_group,Total
0,A & N Islands,2001,Causes,Illness (Aids/STD),Female,0-14,0
1,A & N Islands,2001,Causes,Bankruptcy or Sudden change in Economic,Female,0-14,0
2,A & N Islands,2001,Causes,Cancellation/Non-Settlement of Marriage,Female,0-14,0
3,A & N Islands,2001,Causes,Physical Abuse (Rape/Incest Etc.),Female,0-14,0
4,A & N Islands,2001,Causes,Dowry Dispute,Female,0-14,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237519 entries, 0 to 237518
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   State      237519 non-null  object
 1   Year       237519 non-null  int64 
 2   Type_code  237519 non-null  object
 3   Type       237519 non-null  object
 4   Gender     237519 non-null  object
 5   Age_group  237519 non-null  object
 6   Total      237519 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 12.7+ MB


In [13]:
df.dtypes

State        object
Year          int64
Type_code    object
Type         object
Gender       object
Age_group    object
Total         int64
dtype: object

In [17]:
# get real type of data
def open_object_dtype(s):
    dtypes = set()
    for item in s:
        item_type = type(item);
        if(item_type not in dtypes):
            dtypes.add(item_type)
    return dtypes

for col in df.columns:
    print(open_object_dtype(col))

{<class 'str'>}
{<class 'str'>}
{<class 'str'>}
{<class 'str'>}
{<class 'str'>}
{<class 'str'>}
{<class 'str'>}


##### Vì hài lòng với data nên sẽ không tiền xử lý data

## Tiếp tục khám phá dữ liệu

Em sẽ vừa khám phá dữ liệu và từ những khám phá đưa ra những câu hỏi mới
Đầu tiên ta sẽ tìm hiểu ý nghĩa các cột.  
State, Year, Gender, Age_group
